# 03_anchor_generator
**This file used to generate anchors for UECFOOD100 dataset through kmeans.
Distance here is not Euclidean distance but IoU (Intersection over Union) in
accordance with original yolo-v2 paper.**

In [26]:
import os
import random

import matplotlib.pyplot as plt
import numpy as np

**k-means clustering**:  
- Input: k, set of points [w1,h1], [w2,h2], [w3, h3], ..., [wn, hn] 
- Place centroids c1, ..., ck at random locations (randomly select k [w,h] among input img wh)
- Repeat until convergence:
    - for each point [wi,hi]:
        - find nearest centroid cj. (np.argmin(new_distances, axis=1))
        - assign the point [wi,hi] to cluster j.
    - for each cluster j=1, ...,k: (update centroids)
        - new centroid cj=mean of all points [wi,hi] assigned to cluster j in previous step
        $\frac{1}{n_j}\sum\limits_{x_i\rightarrow c_j}x_i(a)$  (a means a particular attribute in this case a is wh[i])
- Stop when none of the cluster assignments change

In [27]:
def kmeans(wh, centroids, anchor_txt):
    num = wh.shape[0]  # total number of different wh pairs
    k, dim = centroids.shape
    iter = 0
    old_distances = np.zeros((num, k))
    _assignments = -np.ones(num)

    # iterate until
    while True:
        new_distances = []
        iter += 1
        for i in range(num):
            distance = 1 - IoU(wh[i], centroids)  # high IoU represents low distance
            new_distances.append(distance)
        new_distances = np.array(new_distances)
        print('Iter {}: distances: {}'.format(iter, np.sum((np.abs(old_distances - new_distances)))))

        # for each input img assign a centroid (select the closed one)
        assignments = np.argmin(new_distances, axis=1)
        if (assignments == _assignments).all():
            print('final centroids =', centroids)
            save_anchors(centroids, anchor_txt, wh_in_mobilenet)
            return centroids
        else:
            centroid_sums = np.zeros((k, dim), np.float)
            for i in range(num):
                centroid_sums[assignments[i]] += wh[i]  # sum up attribute
            for j in range(k):
                # new centroids
                centroids[j] = centroid_sums[j] / np.sum(assignments == j)

            _assignments = assignments.copy()
            old_distances = new_distances.copy()

In [28]:
def save_anchors(centroids, anchor_txt, wh_in_mobilenet):
    width_in_yolov2 = wh_in_mobilenet[0]
    height_in_yolov2 = wh_in_mobilenet[1]
    with open(anchor_txt, 'w') as file:
        anchors = centroids.copy()
        for i in range(anchors.shape[0]):
            anchors[i][0] *= width_in_yolov2 / 32.
            anchors[i][1] *= height_in_yolov2 / 32.
        widths = anchors[:, 0]
        sorted_indices = np.argsort(widths)  # return the indices that sort tht array
        print('anchors = ', anchors[sorted_indices])

        for i in sorted_indices:
            file.write('%0.2f, %0.2f\n' % (anchors[i, 0], anchors[i, 1]))

In [29]:
def avgIoU(wh, centroids):
    sum = 0.
    for i in range(wh.shape[0]):
        sum += max(IoU(wh[i], centroids))
    return sum / wh.shape[0]

**IoU - Intersection over Union **

In [30]:
def IoU(whi, centroids):
    """ Calculate IoU between current centroids with one in wh array to check if current
    centroids are suitable enough
    :param whi:
    :param centroids:
    :return:
    """
    IOU = []
    for centroid in centroids:
        c_w, c_h = centroid
        w, h = whi
        if c_w >= w and c_h >= h:
            iou = w * h / (c_w * c_h)
        elif c_w >= w and c_h <= h:
            iou = w * c_h / (w * h + (c_w - w) * c_h)
        elif c_w <= w and c_h >= h:
            iou = c_w * h / (w * h + (c_h - h) * c_w)
        else:
            iou = c_w * c_h / (w * h)
        IOU.append(iou)
    return np.array(IOU)

In [51]:
def coordinate2wh(coordinates, shape):
    coordinates = list(map(float, coordinates))
    w = (coordinates[2] - coordinates[0]) / shape[0] # x2-x1
    h = (coordinates[3] - coordinates[1]) / shape[1] # y2-y1
    return w, h

In [55]:
def gen_anchors(n_clusters):
#     dataset_disk = '/Volumes/JS/UECFOOD100_448/'
#     dataset_disk = '/Volumes/JS/UECFOOD256/UECFOOD256/'
    output_path = './'
    train_uec100 = './rice_boxes.txt'

    if not os.path.exists(output_path):
        os.mkdir(output_path)

    wh = []

    with open(train_uec100, 'r') as file:
        for i, line in enumerate(file):
            line = line.rstrip('\n')[1:-1]
            line = line.split(' ')
            coordinates = [int(x) for x in line]
            w, h = coordinate2wh(coordinates, shape=[224, 224])
            wh.append([w, h])
        wh = np.array(wh)

        if n_clusters == 0:  # make from 1 to 10 clusters and pick the best one
            avgIou = []
            for n_cluster in range(1, 11):
                anchor_txt = os.path.join(output_path, 'anchors_%d.txt' % (n_cluster))
                # randomly select n_cluster anchors from wh array which contain w,h for each img
                indices = [random.randrange(wh.shape[0]) for i in range(n_cluster)]
                centroids = wh[indices]
                centroids = kmeans(wh, centroids, anchor_txt)
                avgIou.append([n_cluster, avgIoU(wh, centroids)])
            avgIou = np.array(avgIou)
            plt.plot(avgIou[:, 0], avgIou[:, 1])
            plt.scatter(avgIou[:, 0], avgIou[:, 1], c='r')
            plt.xlabel('number of cluster')
            plt.ylabel('average IoU')
            plt.savefig('avg_iou')
            plt.show()
        else:
            anchor_txt = os.path.join(output_path, 'anchors_%d.txt' % (n_clusters))
            # randomly select n_cluster anchors from wh array which contain w,h for each img
            indices = [random.randrange(wh.shape[0]) for i in range(n_clusters)]
            centroids = wh[indices]
            kmeans(wh, centroids, anchor_txt)

        print('Done!')

In [60]:
wh_in_mobilenet = [224, 224]
# uec256_dims = [800, 600]  # dataset image width=800, height=600
n_clusters = 0

In [61]:
gen_anchors(n_clusters)

Iter 1: distances: 26.11065976113744
Iter 2: distances: 8.972150063207895
final centroids = [[0.58928571 0.58296833]]
anchors =  [[4.125     4.0807783]]
Iter 1: distances: 50.725768930996324
Iter 2: distances: 5.867238501937344
final centroids = [[0.77955665 0.72444581]
 [0.359375   0.41201637]]
anchors =  [[2.515625   2.88411458]
 [5.45689655 5.07112069]]
Iter 1: distances: 84.12731152972296
Iter 2: distances: 12.62935063139469
Iter 3: distances: 5.56828832348238
Iter 4: distances: 3.975442198494473
Iter 5: distances: 1.579449016955533
Iter 6: distances: 1.9713220206311517
Iter 7: distances: 3.1797305485740646
final centroids = [[0.86504121 0.85508242]
 [0.359375   0.41201637]
 [0.71010045 0.61830357]]
anchors =  [[2.515625   2.88411458]
 [4.97070313 4.328125  ]
 [6.05528846 5.98557692]]
Iter 1: distances: 112.12791053163917
Iter 2: distances: 18.196903656770587
Iter 3: distances: 4.0347315517110705
Iter 4: distances: 5.219617493816851
Iter 5: distances: 4.597186990912638
Iter 6: dist

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in true_divide



Iter 105: distances: nan
Iter 106: distances: nan
Iter 107: distances: nan
Iter 108: distances: nan
Iter 109: distances: nan
Iter 110: distances: nan
Iter 111: distances: nan
Iter 112: distances: nan
Iter 113: distances: nan
Iter 114: distances: nan
Iter 115: distances: nan
Iter 116: distances: nan
Iter 117: distances: nan
Iter 118: distances: nan
Iter 119: distances: nan
Iter 120: distances: nan
Iter 121: distances: nan
Iter 122: distances: nan
Iter 123: distances: nan
Iter 124: distances: nan
Iter 125: distances: nan
Iter 126: distances: nan
Iter 127: distances: nan
Iter 128: distances: nan
Iter 129: distances: nan
Iter 130: distances: nan
Iter 131: distances: nan
Iter 132: distances: nan
Iter 133: distances: nan
Iter 134: distances: nan
Iter 135: distances: nan
Iter 136: distances: nan
Iter 137: distances: nan
Iter 138: distances: nan
Iter 139: distances: nan
Iter 140: distances: nan
Iter 141: distances: nan
Iter 142: distances: nan
Iter 143: distances: nan
Iter 144: distances: nan

Iter 439: distances: nan
Iter 440: distances: nan
Iter 441: distances: nan
Iter 442: distances: nan
Iter 443: distances: nan
Iter 444: distances: nan
Iter 445: distances: nan
Iter 446: distances: nan
Iter 447: distances: nan
Iter 448: distances: nan
Iter 449: distances: nan
Iter 450: distances: nan
Iter 451: distances: nan
Iter 452: distances: nan
Iter 453: distances: nan
Iter 454: distances: nan
Iter 455: distances: nan
Iter 456: distances: nan
Iter 457: distances: nan
Iter 458: distances: nan
Iter 459: distances: nan
Iter 460: distances: nan
Iter 461: distances: nan
Iter 462: distances: nan
Iter 463: distances: nan
Iter 464: distances: nan
Iter 465: distances: nan
Iter 466: distances: nan
Iter 467: distances: nan
Iter 468: distances: nan
Iter 469: distances: nan
Iter 470: distances: nan
Iter 471: distances: nan
Iter 472: distances: nan
Iter 473: distances: nan
Iter 474: distances: nan
Iter 475: distances: nan
Iter 476: distances: nan
Iter 477: distances: nan
Iter 478: distances: nan


Iter 866: distances: nan
Iter 867: distances: nan
Iter 868: distances: nan
Iter 869: distances: nan
Iter 870: distances: nan
Iter 871: distances: nan
Iter 872: distances: nan
Iter 873: distances: nan
Iter 874: distances: nan
Iter 875: distances: nan
Iter 876: distances: nan
Iter 877: distances: nan
Iter 878: distances: nan
Iter 879: distances: nan
Iter 880: distances: nan
Iter 881: distances: nan
Iter 882: distances: nan
Iter 883: distances: nan
Iter 884: distances: nan
Iter 885: distances: nan
Iter 886: distances: nan
Iter 887: distances: nan
Iter 888: distances: nan
Iter 889: distances: nan
Iter 890: distances: nan
Iter 891: distances: nan
Iter 892: distances: nan
Iter 893: distances: nan
Iter 894: distances: nan
Iter 895: distances: nan
Iter 896: distances: nan
Iter 897: distances: nan
Iter 898: distances: nan
Iter 899: distances: nan
Iter 900: distances: nan
Iter 901: distances: nan
Iter 902: distances: nan
Iter 903: distances: nan
Iter 904: distances: nan
Iter 905: distances: nan


...

Iter 12964: distances: nan
Iter 12965: distances: nan
Iter 12966: distances: nan
Iter 12967: distances: nan
Iter 12968: distances: nan
Iter 12969: distances: nan
Iter 12970: distances: nan
Iter 12971: distances: nan
Iter 12972: distances: nan
Iter 12973: distances: nan
Iter 12974: distances: nan
Iter 12975: distances: nan
Iter 12976: distances: nan
Iter 12977: distances: nan
Iter 12978: distances: nan
Iter 12979: distances: nan
Iter 12980: distances: nan
Iter 12981: distances: nan
Iter 12982: distances: nan
Iter 12983: distances: nan
Iter 12984: distances: nan
Iter 12985: distances: nan
Iter 12986: distances: nan
Iter 12987: distances: nan
Iter 12988: distances: nan
Iter 12989: distances: nan
Iter 12990: distances: nan
Iter 12991: distances: nan
Iter 12992: distances: nan
Iter 12993: distances: nan
Iter 12994: distances: nan
Iter 12995: distances: nan
Iter 12996: distances: nan
Iter 12997: distances: nan
Iter 12998: distances: nan
Iter 12999: distances: nan
Iter 13000: distances: nan
I

Iter 13364: distances: nan
Iter 13365: distances: nan
Iter 13366: distances: nan
Iter 13367: distances: nan
Iter 13368: distances: nan
Iter 13369: distances: nan
Iter 13370: distances: nan
Iter 13371: distances: nan
Iter 13372: distances: nan
Iter 13373: distances: nan
Iter 13374: distances: nan
Iter 13375: distances: nan
Iter 13376: distances: nan
Iter 13377: distances: nan
Iter 13378: distances: nan
Iter 13379: distances: nan
Iter 13380: distances: nan
Iter 13381: distances: nan
Iter 13382: distances: nan
Iter 13383: distances: nan
Iter 13384: distances: nan
Iter 13385: distances: nan
Iter 13386: distances: nan
Iter 13387: distances: nan
Iter 13388: distances: nan
Iter 13389: distances: nan
Iter 13390: distances: nan
Iter 13391: distances: nan
Iter 13392: distances: nan
Iter 13393: distances: nan
Iter 13394: distances: nan
Iter 13395: distances: nan
Iter 13396: distances: nan
Iter 13397: distances: nan
Iter 13398: distances: nan
Iter 13399: distances: nan
Iter 13400: distances: nan
I

Iter 13820: distances: nan
Iter 13821: distances: nan
Iter 13822: distances: nan
Iter 13823: distances: nan
Iter 13824: distances: nan
Iter 13825: distances: nan
Iter 13826: distances: nan
Iter 13827: distances: nan
Iter 13828: distances: nan
Iter 13829: distances: nan
Iter 13830: distances: nan
Iter 13831: distances: nan
Iter 13832: distances: nan
Iter 13833: distances: nan
Iter 13834: distances: nan
Iter 13835: distances: nan
Iter 13836: distances: nan
Iter 13837: distances: nan
Iter 13838: distances: nan
Iter 13839: distances: nan
Iter 13840: distances: nan
Iter 13841: distances: nan
Iter 13842: distances: nan
Iter 13843: distances: nan
Iter 13844: distances: nan
Iter 13845: distances: nan
Iter 13846: distances: nan
Iter 13847: distances: nan
Iter 13848: distances: nan
Iter 13849: distances: nan
Iter 13850: distances: nan
Iter 13851: distances: nan
Iter 13852: distances: nan
Iter 13853: distances: nan
Iter 13854: distances: nan
Iter 13855: distances: nan
Iter 13856: distances: nan
I

Iter 14213: distances: nan
Iter 14214: distances: nan
Iter 14215: distances: nan
Iter 14216: distances: nan
Iter 14217: distances: nan
Iter 14218: distances: nan
Iter 14219: distances: nan
Iter 14220: distances: nan
Iter 14221: distances: nan
Iter 14222: distances: nan
Iter 14223: distances: nan
Iter 14224: distances: nan
Iter 14225: distances: nan
Iter 14226: distances: nan
Iter 14227: distances: nan
Iter 14228: distances: nan
Iter 14229: distances: nan
Iter 14230: distances: nan
Iter 14231: distances: nan
Iter 14232: distances: nan
Iter 14233: distances: nan
Iter 14234: distances: nan
Iter 14235: distances: nan
Iter 14236: distances: nan
Iter 14237: distances: nan
Iter 14238: distances: nan
Iter 14239: distances: nan
Iter 14240: distances: nan
Iter 14241: distances: nan
Iter 14242: distances: nan
Iter 14243: distances: nan
Iter 14244: distances: nan
Iter 14245: distances: nan
Iter 14246: distances: nan
Iter 14247: distances: nan
Iter 14248: distances: nan
Iter 14249: distances: nan
I

KeyboardInterrupt: 

In [39]:
train_uec100 = '/Users/jianingsun/Documents/Research/Nov/mask_food/example/rice/rice_boxes.txt'
with open(train_uec100, 'r') as file:
    for i, line in enumerate(file):
        line = line.rstrip('\n')[1:-1]
        line = line.split(' ')
        coordinates = [int(x) for x in line]
        print(coordinates)

[12, 74, 93, 161]
[9, 90, 156, 202]
[6, 63, 69, 151]
[1, 89, 76, 196]
[30, 33, 184, 205]
[1, 22, 96, 144]
[13, 116, 103, 222]
[21, 10, 219, 161]
[1, 88, 71, 174]
[22, 13, 150, 159]
[48, 79, 172, 175]
[10, 30, 223, 147]
[12, 79, 116, 192]
[15, 10, 200, 166]
[14, 60, 103, 144]
[14, 87, 219, 212]
[89, 134, 150, 213]
[123, 87, 197, 163]
[145, 35, 210, 129]
[1, 2, 173, 204]
[10, 67, 82, 128]
[10, 17, 193, 217]
[36, 21, 180, 194]
[145, 45, 221, 129]
[122, 130, 171, 186]
[2, 65, 220, 172]
[15, 1, 222, 218]
[29, 15, 191, 137]
[25, 28, 173, 211]
[100, 57, 223, 178]
[12, 99, 94, 201]
[2, 13, 219, 223]
[12, 108, 92, 205]
[29, 19, 222, 221]
[1, 26, 223, 198]
[1, 3, 223, 222]
[5, 6, 169, 154]
[1, 19, 223, 207]
[34, 23, 207, 216]
[1, 90, 97, 201]
[36, 36, 182, 177]
[57, 43, 174, 183]
[49, 22, 203, 173]
[30, 57, 175, 203]
[28, 16, 205, 213]
[34, 3, 191, 161]
[30, 46, 198, 183]
[19, 89, 94, 182]
[22, 118, 103, 214]
[4, 113, 85, 203]
[21, 119, 92, 208]
[10, 111, 103, 210]
[14, 117, 99, 216]
